In [ ]:
import seaborn as sns
import numpy as np
from scipy import stats
from baynes.plotter import FitPlotter
from baynes.model_utils import get_model

# Devil's funnel
### The centered parametrization produces divergences

In [ ]:
model = get_model('funnel.stan')
fit = model.sample({},
                   chains=4,
                   iter_warmup=1000,
                   iter_sampling=300,
                   save_warmup=True,
                   show_progress=True,
                   adapt_delta=0.9)
print(fit.diagnose())

fplot = FitPlotter()

xmax=40
ymin, ymax = -4, 5
X = np.linspace(-xmax, xmax, 1001)
Y = np.linspace(ymin, ymax, 1001)
Z=[]
for y in Y:
    Z.append(stats.norm.pdf(X, 0, np.exp(y))* stats.norm.pdf(y, loc=0, scale=3))
Z = np.array(Z)
levels = Z[[900, 850, 800, 750, 700, 650, 600], 500]
ax = fplot.new_figure('funnel').subplots()
ax.contour(X, Y, Z, levels=levels, colors='grey', alpha=0.5, zorder=0)
df = fit.draws_pd()
sns.scatterplot(df, x='x', y='y', legend=False, ax=ax)
sns.scatterplot(df.loc[df.divergent__==True], x='x', y='y',  legend=False, ax=ax)


ax.set_xlim(-xmax, xmax)
ax.set_ylim(ymin,ymax)
fplot.resize(5,5)

### The noncentered parametrization works correctly because Stan is actually sampling $z$ and $y$ from a normal distribution

In [ ]:
model = get_model('funnel_noncentered.stan')
print(model.code())
fit = model.sample({},
                   chains=4,
                   iter_warmup=1000,
                   iter_sampling=300,
                   save_warmup=True,
                   show_progress=True,
                   adapt_delta=0.9)
print(fit.diagnose())

Z=[]
for y in Y:
    Z.append(stats.norm.pdf(X, 0, np.exp(y))* stats.norm.pdf(y, loc=0, scale=3))
Z = np.array(Z)
levels=Z[[900, 850, 800, 750, 700, 650, 600], 500]
ax = fplot.new_figure('funnel').subplots()
ax.contour(X, Y, Z, levels=levels, colors='grey', alpha=0.5, zorder=0)
df= fit.draws_pd()
sns.scatterplot(df, x='x', y='y', legend=False, ax=ax)
sns.scatterplot(df.loc[df.divergent__==True], x='x', y='y',  legend=False, ax=ax)
ax.set_xlim(-xmax, xmax)
ax.set_ylim(ymin,ymax)
fplot.resize(5,5)

In [ ]:
xmax=4
ymin, ymax = -8.5, 8.5
X = np.linspace(-xmax, xmax, 1001)
Y= np.linspace(ymin, ymax, 1001)
Z=[]
for y in Y:
    Z.append(stats.norm.pdf(X, 0, 1)* stats.norm.pdf(y, loc=0, scale=3))
Z = np.array(Z)
levels=Z[[1000,900, 800, 750, 700, 650, 600], 500]
ax = fplot.new_figure('funnel').subplots()
ax.contour(X, Y, Z, levels=levels, colors='grey', alpha=0.5, zorder=0)
df = fit.draws_pd()
sns.scatterplot(df, x='z', y='y', legend=False, ax=ax)
sns.scatterplot(df.loc[df.divergent__==True], x='z', y='y',  legend=False, ax=ax)
ax.set_xlim(-xmax, xmax)
ax.set_ylim(ymin,ymax)
fplot.resize(5,5)